# Azure AI Foundry Response API Demo

This notebook demonstrates how to use Azure AI Foundry's Response API with parallel tool calls.

## Tool Function Stubs

First, let's create sample stub functions that simulate the tools the AI will call.

## Environment Setup

First, let's load the environment variables from the .env file:

### 🔑 How to Get Your Azure AI Foundry Credentials

To use this notebook with real Azure AI Foundry API calls, you need:

1. **Azure AI Foundry Project**: 
   - Go to [Azure AI Foundry](https://ai.azure.com/)
   - Create a new project or use an existing one

2. **Get Your Endpoint**:
   - In your AI Foundry project, go to **Settings** → **General**
   - Copy the **Endpoint URL** (format: `https://your-project.region.api.cognitive.microsoft.com`)

3. **Get Your API Key**:
   - In your project settings, go to **Keys and Endpoint**
   - Copy one of the **API Keys**

4. **Update the .env file**:
   - Open the `.env` file in this directory
   - Replace `your-project-name.eastus.api.cognitive.microsoft.com` with your actual endpoint
   - Replace `your-api-key-here` with your actual API key

**Note**: The `.env` file is already configured to be ignored by git for security.

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Check if environment variables are loaded
endpoint = os.getenv("AZURE_AI_FOUNDRY_ENDPOINT")
api_key = os.getenv("AZURE_AI_FOUNDRY_KEY")
model_name = os.getenv("MODEL_DEPLOYMENT_NAME", "gpt-4.1")  # Default to gpt-4.1
api_version = os.getenv("API_VERSION", "2024-10-01-preview")

print("🔧 Environment Configuration:")
print(f"   Endpoint: {'✅ Set' if endpoint else '❌ Missing'}")
print(f"   API Key: {'✅ Set' if api_key else '❌ Missing'}")
print(f"   Model: {model_name}")
print(f"   API Version: {api_version}")

if not endpoint or not api_key:
    print("\n⚠️  To use the real Azure AI Foundry API:")
    print("   1. Open the .env file in this directory")
    print("   2. Replace the placeholder values with your actual:")
    print("      - Azure AI Foundry project endpoint")
    print("      - API key from your project settings")
    print("   3. Re-run this cell")
    print("\n🎭 For now, the notebook will run in demo mode with mock responses.")
else:
    print("\n✅ Ready to use Azure AI Foundry API!")

In [ ]:
import json
import time
from datetime import datetime
from typing import Dict, Any

def query_warehouse(portfolio_id: str) -> Dict[str, Any]:
    """
    Sample stub function that simulates querying a Fabric Warehouse for NAV data.
    
    Args:
        portfolio_id (str): The portfolio ID to query NAV for
        
    Returns:
        Dict containing NAV information
    """
    # Simulate some processing time
    time.sleep(0.5)
    
    # Sample NAV data based on portfolio_id
    sample_nav_data = {
        "portfolio_id": portfolio_id,
        "nav": 125.67,
        "currency": "USD",
        "as_of_date": datetime.now().strftime("%Y-%m-%d"),
        "last_updated": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "total_assets": 15678000.00,
        "total_liabilities": 234000.00,
        "net_assets": 15444000.00,
        "shares_outstanding": 122850,
        "performance": {
            "1_day": 0.45,
            "1_week": 2.34,
            "1_month": 5.67,
            "ytd": 12.45
        },
        "data_source": "Fabric Warehouse",
        "status": "success"
    }
    
    print(f"📊 Queried warehouse for portfolio {portfolio_id}")
    return sample_nav_data

def fetch_crm_advisor(advisor_id: str) -> Dict[str, Any]:
    """
    Sample stub function that simulates fetching advisor details from a CRM system.
    
    Args:
        advisor_id (str): The advisor ID to fetch details for
        
    Returns:
        Dict containing advisor information
    """
    # Simulate some processing time
    time.sleep(0.3)
    
    # Sample advisor data based on advisor_id
    sample_advisor_data = {
        "advisor_id": advisor_id,
        "name": "Sarah Johnson",
        "title": "Senior Financial Advisor",
        "email": "sarah.johnson@company.com",
        "phone": "+1-555-0123",
        "branch": "Downtown Office",
        "specializations": ["Retirement Planning", "Investment Management", "Estate Planning"],
        "certifications": ["CFP", "CFA", "ChFC"],
        "years_experience": 12,
        "client_count": 145,
        "assets_under_management": 85000000.00,
        "rating": 4.8,
        "license_numbers": {
            "series_7": "12345678",
            "series_66": "87654321"
        },
        "availability": {
            "status": "available",
            "next_appointment": "2025-09-23 14:00:00",
            "timezone": "EST"
        },
        "data_source": "CRM System",
        "status": "success"
    }
    
    print(f"👤 Fetched CRM data for advisor {advisor_id}")
    return sample_advisor_data

print("✅ Tool stub functions defined successfully!")

## Test the Tool Functions

Let's test our stub functions to make sure they work properly:

In [ ]:
# Test the warehouse query function
print("🧪 Testing query_warehouse function:")
warehouse_result = query_warehouse("PORTFOLIO_123")
print(json.dumps(warehouse_result, indent=2))

print("\n" + "="*50 + "\n")

# Test the CRM advisor function  
print("🧪 Testing fetch_crm_advisor function:")
advisor_result = fetch_crm_advisor("ADV_456")
print(json.dumps(advisor_result, indent=2))

## Tool Call Handler

Now let's create a handler function that can process tool calls from the AI response:

In [ ]:
def handle_tool_calls(tool_calls):
    """
    Handle tool calls from the AI response by calling the appropriate functions.
    
    Args:
        tool_calls: List of tool calls from the AI response
        
    Returns:
        Dict containing results from all tool calls
    """
    # Registry of available tools
    tool_registry = {
        "query_warehouse": query_warehouse,
        "fetch_crm_advisor": fetch_crm_advisor
    }
    
    results = {}
    
    for tool_call in tool_calls:
        tool_name = tool_call.get("name")
        tool_args = tool_call.get("arguments", {})
        
        if tool_name in tool_registry:
            print(f"🔧 Executing tool: {tool_name} with args: {tool_args}")
            try:
                result = tool_registry[tool_name](**tool_args)
                results[tool_name] = result
            except Exception as e:
                print(f"❌ Error executing {tool_name}: {str(e)}")
                results[tool_name] = {"error": str(e)}
        else:
            print(f"⚠️  Unknown tool: {tool_name}")
            results[tool_name] = {"error": f"Unknown tool: {tool_name}"}
    
    return results

print("✅ Tool call handler defined successfully!")

## Azure AI Foundry API Call

Now let's call the Azure AI Foundry Response API with parallel tool calls enabled:

In [ ]:
import requests

# Use the environment variables loaded in the previous cell
print(f"🚀 Preparing API call to Azure AI Foundry...")
print(f"   Endpoint: {endpoint}")
print(f"   Model: {model_name}")

# Check if we can make real API calls or use demo mode
if not endpoint or not api_key or endpoint == "https://your-project-name.eastus.api.cognitive.microsoft.com":
    print("⚠️  Using demo mode (environment variables not configured)")
    demo_mode = True
else:
    demo_mode = False
    url = f"{endpoint}/openai/responses?api-version={api_version}"
    headers = {
        "Content-Type": "application/json",
        "api-key": api_key
    }

# Define the payload with parallel tool calls
payload = {
    "model": model_name,
    "input": "Get me the latest NAV from Warehouse and the advisor info from CRM",
    "response_config": {
        "parallel_tool_calls": True  # 👈 enables concurrent tool execution
    },
    "tools": [
        {
            "type": "function",
            "name": "query_warehouse",
            "description": "Fetch NAV (Net Asset Value) data from Fabric Warehouse",
            "parameters": {
                "type": "object",
                "properties": {
                    "portfolio_id": {
                        "type": "string",
                        "description": "The unique identifier for the portfolio"
                    }
                },
                "required": ["portfolio_id"]
            }
        },
        {
            "type": "function",
            "name": "fetch_crm_advisor",
            "description": "Fetch advisor details and contact information from CRM system",
            "parameters": {
                "type": "object",
                "properties": {
                    "advisor_id": {
                        "type": "string",
                        "description": "The unique identifier for the advisor"
                    }
                },
                "required": ["advisor_id"]
            }
        }
    ]
}

if demo_mode:
    # Mock response for demo purposes
    print("\n🎭 Demo Mode: Simulating AI response with parallel tool calls")
    mock_response = {
        "choices": [{
            "message": {
                "role": "assistant",
                "content": "I'll help you get the latest NAV from the warehouse and advisor information from the CRM system. Let me fetch both pieces of information for you.",
                "tool_calls": [
                    {
                        "id": "call_warehouse_nav",
                        "type": "function",
                        "name": "query_warehouse",
                        "arguments": {"portfolio_id": "PORTFOLIO_123"}
                    },
                    {
                        "id": "call_crm_advisor", 
                        "type": "function",
                        "name": "fetch_crm_advisor",
                        "arguments": {"advisor_id": "ADV_456"}
                    }
                ]
            },
            "finish_reason": "tool_calls"
        }],
        "usage": {
            "prompt_tokens": 245,
            "completion_tokens": 58,
            "total_tokens": 303
        }
    }
    
    print("📝 Mock AI Response:")
    print(f"   Content: {mock_response['choices'][0]['message']['content']}")
    print(f"   Tool Calls: {len(mock_response['choices'][0]['message']['tool_calls'])}")
    
    # Execute the tool calls
    tool_calls = mock_response["choices"][0]["message"]["tool_calls"]
    print(f"\n🔧 Executing {len(tool_calls)} tool calls in parallel...")
    tool_results = handle_tool_calls(tool_calls)
    
    print("\n🎯 Tool execution results:")
    print(json.dumps(tool_results, indent=2))
    
else:
    # Real API call to Azure AI Foundry
    print(f"\n🌐 Making real API call to: {url}")
    try:
        response = requests.post(url, headers=headers, json=payload, timeout=30)
        response.raise_for_status()
        
        api_response = response.json()
        print("✅ API Response received successfully!")
        print(f"   Status Code: {response.status_code}")
        
        # Display the AI's response
        if "choices" in api_response and api_response["choices"]:
            message = api_response["choices"][0].get("message", {})
            print(f"\n🤖 AI Response: {message.get('content', 'No content')}")
            
            # Check if there are tool calls to execute
            if "tool_calls" in message:
                tool_calls = message["tool_calls"]
                print(f"\n🔧 Executing {len(tool_calls)} tool calls...")
                tool_results = handle_tool_calls(tool_calls)
                print("\n🎯 Tool execution results:")
                print(json.dumps(tool_results, indent=2))
            else:
                print("ℹ️  No tool calls in the response")
        
        # Display usage information if available
        if "usage" in api_response:
            usage = api_response["usage"]
            print(f"\n📊 Token Usage:")
            print(f"   Prompt: {usage.get('prompt_tokens', 'N/A')}")
            print(f"   Completion: {usage.get('completion_tokens', 'N/A')}")
            print(f"   Total: {usage.get('total_tokens', 'N/A')}")
        
    except requests.exceptions.HTTPError as e:
        print(f"❌ HTTP Error: {e}")
        print(f"   Response: {response.text if 'response' in locals() else 'No response'}")
    except requests.exceptions.Timeout:
        print("❌ Request timed out after 30 seconds")
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {str(e)}")
    except json.JSONDecodeError:
        print("❌ Failed to parse JSON response")
    except Exception as e:
        print(f"❌ Unexpected error: {str(e)}")